### 5.2 Whole Slide Scoring - Prediction Confidence Segmentation

Prediction confidence heatmaps were segmented to generate the CNN model-based whole-slide scores. 

First, a CNN confidence threshold was applied to the heatmaps, with only prediction confidences higher than the threshold retained, indicating the existences of plaques predicted with high confidence. Morphological opening and closing operations were then performed to smooth the masks, and prediction areas exceeding a size threshold set to eliminate CNN-noise provided the predicted quantities of Aβ pathologies (ie. cored plaque, diffuse plaque, and CAA). 

These two hyperparameters, CNN confidence threshold (core: 0.1; diffuse: 0.95; CAA: 0.9) and size threshold (cored: 100; diffuse: 1; CAA: 200), were optimized exclusively by statistical analysis on the combined training and validation sets (32 WSIs). These were then used on the hold-out test set with 30 unseen WSIs to show the generalizability. 

In [1]:
import csv
import glob, os

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import measure
from scipy import stats

from tqdm import tqdm

In [27]:
CSV_DIR = 'data/outputs/CNNscore/WSI_CERAD_AREA.csv'
HEATMAP_DIR = 'data/outputs/heatmaps'
HEATMAP_DIR = 'data/outputs/heatmaps_stride16/'
SAVE_DIR = 'data/outputs/CNNscore/'

In [19]:
# IMG_DIR = 'data/norm_tiles'
filenames = sorted(os.listdir(HEATMAP_DIR))
filenames = [os.path.splitext(file)[0] for file in filenames]
print(filenames)

['NA3777-02_AB', 'NA4077-02_AB', 'NA4092-02_AB', 'NA4107-02_AB', 'NA4160-02_AB', 'NA4195-02_AB', 'NA4256-02_AB', 'NA4299-02_AB', 'NA4391-02_AB', 'NA4450-02_AB', 'NA4463-02_AB', 'NA4471-02_AB', 'NA4553-02_AB', 'NA4626-02_AB', 'NA4672-02_AB', 'NA4675-02_AB', 'NA4691-02_AB', 'NA4695-02_AB']


In [22]:
test = pd.DataFrame({"WSI_ID": filenames})
test.to_csv(CSV_DIR, index=False)

In [23]:
test = pd.read_csv(CSV_DIR)
print(test)

          WSI_ID
0   NA3777-02_AB
1   NA4077-02_AB
2   NA4092-02_AB
3   NA4107-02_AB
4   NA4160-02_AB
5   NA4195-02_AB
6   NA4256-02_AB
7   NA4299-02_AB
8   NA4391-02_AB
9   NA4450-02_AB
10  NA4463-02_AB
11  NA4471-02_AB
12  NA4553-02_AB
13  NA4626-02_AB
14  NA4672-02_AB
15  NA4675-02_AB
16  NA4691-02_AB
17  NA4695-02_AB


In [3]:
if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

In [24]:
file = pd.read_csv(CSV_DIR)
filenames = list(file['WSI_ID'])
img_class = ['cored', 'diffuse', 'caa']

# two hyperparameters
confidence_thresholds = [0.1, 0.95, 0.9]
pixel_thresholds = [100, 1, 200]

In [69]:
# UserWarning: The argument 'neighbors' is deprecated and will be removed in scikit-image 0.18,
# use 'connectivity' instead. For neighbors=8, use connectivity=2
#   This is separate from the ipykernel package so we can avoid doing imports until

def count_blobs(mask,
               threshold=1500):
    labels = measure.label(mask, neighbors=8, background=0)
    labels = measure.label(mask, connectivity=2, background=0)
    img_mask = np.zeros(mask.shape, dtype='uint8')
    labeled_mask = np.zeros(mask.shape, dtype='uint16')
    sizes = []
    locations = []
    
    # loop over the unique components
    for label in np.unique(labels):
        # if this is the background label, ignore it
        if label == 0:
            continue
        # otherwise, construct the label mask and count the
        # number of pixels 
        labelMask = np.zeros(mask.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero(labelMask)
        
        # if the number of pixels in the component is sufficiently
        # large, then add it to our mask of "large blobs"
        if numPixels > threshold:
            sizes.append(numPixels)
            img_mask = cv2.add(img_mask, labelMask)
            
            # Save confirmed unique location of plaque
            labeled_mask[labels==label] = label
#             location = np.where(labelMask == 255)
#             locations.append(location)

#             row_loc, col_loc = np.where(labelMask == 255)
#             print("row_loc, col_loc", (row_loc, col_loc))

    return sizes, img_mask, labeled_mask

In [43]:
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
def plot_mask(mask_array) :
    fig = plt.figure(figsize=(45,15))
#     ax = plt.figure(figsize=(45,15))

    ax = fig.add_subplot(111)

    im = ax.imshow(mask_array, cmap=plt.cm.get_cmap('viridis', 20), vmin=0, vmax=1)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax, ticks=[0.0, 0.25, 0.5, 0.75, 1.0])
    
    return plt.show()

In [75]:
from PIL import Image
def saveMask(mask_array, save_dir) :
    
    mask_array = np.repeat(mask_array[:,:, np.newaxis], 3, axis=2)
    
    # mask_array[:,:,0] = RED, mask_array[:,:,1] = Green, mask_array[:,:,2] = Blue
    idx = np.where(mask_array[:,:,0] == 255)  # Index of label 1 (WM)

    # For label 0, leave as black color
    # For label 1, set to cyan color: R0G255B255
    mask_array[:,:,0].flat[np.ravel_multi_index(idx, mask_array[:,:,0].shape)] = 0
    mask_array[:,:,1].flat[np.ravel_multi_index(idx, mask_array[:,:,1].shape)] = 255
    mask_array[:,:,2].flat[np.ravel_multi_index(idx, mask_array[:,:,2].shape)] = 255
#     # For label 2, set to yellow color: R255G255B0
#     mask_array[:,:,0].flat[np.ravel_multi_index(idx_2, mask_array[:,:,0].shape)] = 255
#     mask_array[:,:,1].flat[np.ravel_multi_index(idx_2, mask_array[:,:,1].shape)] = 255
#     mask_array[:,:,2].flat[np.ravel_multi_index(idx_2, mask_array[:,:,2].shape)] = 0

    mask_array = mask_array.astype(np.uint8) # PIL save only accepts uint8 {0,..,255}
    save_img = Image.fromarray(mask_array, 'RGB')
    save_img.save(save_dir)
    print("Saved at: " + save_dir)

In [ ]:
def locatePlaque(mask_array, seg_array) :
    
    idx = np.where(mask_array == 255)
    seg_array[idx]

In [78]:
BRAINSEG_NP_DIR = 'data/brainseg/numpy/'
SAVE_IMG_DIR = 'data/outputs/masked_plaque/images/'
SAVE_NP_DIR = 'data/outputs/masked_plaque/numpy/'
print(os.listdir(BRAINSEG_NP_DIR))

['NA4672-02_AB.npy', 'NA4675-02_AB.npy', 'NA4691-02_AB.npy', 'NA4695-02_AB.npy', 'NA4107-02_AB.npy', 'NA4463-02_AB.npy', 'NA4391-02_AB.npy', 'NA4160-02_AB.npy', 'NA4471-02_AB.npy', 'NA4626-02_AB.npy', 'NA3777-02_AB.npy', 'NA4553-02_AB.npy', 'NA4299-02_AB.npy', 'NA4092-02_AB.npy', 'NA4077-02_AB.npy', 'NA4195-02_AB.npy', 'NA4256-02_AB.npy', 'NA4450-02_AB.npy']


In [76]:
new_file = file
for index in [0,1,2]:
    preds = np.zeros(len(file))
    confidence_threshold = confidence_thresholds[index]
    pixel_threshold = pixel_thresholds[index]

    for i, WSIname in tqdm(enumerate(filenames)):
        try:
            heatmap_path = HEATMAP_DIR+'new_WSI_heatmap_{}.npy'.format(WSIname)
            h = np.load(heatmap_path)

        except:
            heatmap_path = HEATMAP_DIR+'{}.npy'.format(WSIname)
            h = np.load(heatmap_path)
            seg_path = BRAINSEG_NP_DIR+'{}.npy'.format(WSIname)
            seg = np.load(seg_path)

        mask = h[index] > confidence_threshold
        mask = mask.astype(np.float32)
        
#         print("Initial Mask:", mask)
#         plot_mask(mask)

        kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

        # Apply morphological closing, then opening operations 
        opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        
#         print("Denoised Mask:", closing)
#         print("Denoised Mask Shape: ", closing.shape)
#         plot_mask(closing)

        labels, img_mask, labeled_mask = count_blobs(closing, threshold=pixel_threshold)
    
        save_img = SAVE_IMG_DIR + WSIname \
                    + "_" + img_class[index] + ".png"
        save_np = SAVE_NP_DIR + WSIname \
                    + "_" + img_class[index] + ".npy"
        np.save(save_np, labeled_mask)
        saveMask(img_mask, save_img)
        

#         print(labels)
#         print(img_mask.shape)
        preds[i] = len(labels)

    print(confidence_threshold, pixel_threshold)

    new_file['CNN_{}_count'.format(img_class[index])] = preds

# new_file.to_csv(SAVE_DIR+'CNN_vs_CERAD.csv', index=False)

0it [00:00, ?it/s]/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The argument 'neighbors' is deprecated and will be removed in scikit-image 0.18, use 'connectivity' instead. For neighbors=8, use connectivity=2
  import sys
0it [00:07, ?it/s]


KeyboardInterrupt: 

In [65]:
print(np.where(np.asarray(a) == 172))
print(np.where(np.asarray(m) == 255))

(array([ 0, 70]),)
(array([ 153,  154,  154, ..., 2718, 2718, 2718]), array([2081, 2080, 2081, ..., 2861, 2862, 2863]))


In [79]:
save_img = SAVE_IMG_DIR + WSIname \
                    + "_" + img_class[index] + ".png"
save_np = SAVE_NP_DIR + WSIname \
                    + "_" + img_class[index] + ".npy"
# np.save(save_np, labeled_mask)
saveMask(img_mask, save_img)

Saved at: data/outputs/masked_plaque/images/NA3777-02_AB_cored.png


In [ ]:
# Original Code
def count_blobs(mask,
               threshold=1500):
    labels = measure.label(mask, neighbors=8, background=0)
    new_mask = np.zeros(mask.shape, dtype='uint8')
    sizes = []
    
    # loop over the unique components
    for label in np.unique(labels):
        # if this is the background label, ignore it
        if label == 0:
            continue
        # otherwise, construct the label mask and count the
        # number of pixels 
        labelMask = np.zeros(mask.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero(labelMask)
        
        # if the number of pixels in the component is sufficiently
        # large, then add it to our mask of "large blobs"
        if numPixels > threshold:
            sizes.append(numPixels)
            new_mask = cv2.add(new_mask, labelMask)

    return sizes, new_mask

In [ ]:
# Original Code
new_file = file
for index in [0,1,2]:
    preds = np.zeros(len(file))
    confidence_threshold = confidence_thresholds[index]
    pixel_threshold = pixel_thresholds[index]

    for i, WSIname in tqdm(enumerate(filenames)):
        try:
            heatmap_path = HEATMAP_DIR+'new_WSI_heatmap_{}.npy'.format(WSIname)
            h = np.load(heatmap_path)

        except:
            heatmap_path = HEATMAP_DIR+'{}.npy'.format(WSIname)
            h = np.load(heatmap_path)

        mask = h[index] > confidence_threshold
        mask = mask.astype(np.float32)

        kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

        # Apply morphological closing, then opening operations 
        opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

        a, m = count_blobs(closing, threshold=pixel_threshold)

        preds[i] = len(a)

    print(confidence_threshold, pixel_threshold)

    new_file['CNN_{}_count'.format(img_class[index])] = preds

new_file.to_csv(SAVE_DIR+'CNN_vs_CERAD.csv', index=False)

In [6]:
new_file

NameError: name 'new_file' is not defined